## Tall Format Trait Table Season 4

In [ ]:
import datetime
import numpy as np
import pandas as pd

In [ ]:
df_0 = pd.read_csv('data/raw/mac_season_4.csv', low_memory=False)
df_0.shape

In [ ]:
# df_0.trait.unique()

In [ ]:
want_to_print_traits = False

if want_to_print_traits:

    for trait in df_0.trait.unique():
        print(f'Unit for {trait}: {df_0.loc[df_0.trait == trait].units.unique()}')

In [ ]:
# df_0.columns

In [ ]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id', 'treatment_id', 'city', 
                'scientificname', 'commonname', 'genus', 'species_id', 'cultivar_id', 'author', 'citation_year',
                'treatment', 'time', 'raw_date', 'month', 'year', 'dateloc', 'trait_description', 'units', 'n', 'statname',
                'stat', 'notes', 'access_level', 'entity', 'method_name', 'view_url', 'edit_url']

In [ ]:
df_1 = df_0.drop(labels=cols_to_drop, axis=1)
# df_1.head()

In [ ]:
# df_1.shape

In [ ]:
# print(df_1.date.min())
# print(df_1.date.max())

### I. Change date values to iso date format
* date(s) as index?

In [ ]:
new_dates = []

for d in df_1.date.values:
    
    if 'Phoenix' in d:
        new_name = d[:-18]
        new_dates.append(new_name)
    
    else:
        new_name = d
        new_dates.append(new_name)
        
print(len(new_dates))

In [ ]:
iso_format_dates = pd.to_datetime(new_dates)

In [ ]:
df_1['date_1'] = iso_format_dates
# df_1.head()

### II. Change sitenames to strip E and W subplot designations
* Check which traits are associated with subplots

In [ ]:
e_w_plots = df_1.loc[(df_1.sitename.str.endswith(' E')) | (df_1.sitename.str.endswith(' W'))]
e_w_plots.trait.unique()

In [ ]:
e_w_plot_traits = e_w_plots.trait.unique()

In [ ]:
sitename_values = df_1.sitename.values
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))

In [ ]:
df_1['sitename_1'] = no_e_w_names
# df_1.sample(n=7)

In [ ]:
# df_1.loc[df_1.sitename.str.endswith(' E')].iloc[0]

#### III. Extract Range and Column Values

In [ ]:
df_1['range'] = df_1['sitename'].str.extract("Range (\d+)").astype(int)
df_1['column'] = df_1['sitename'].str.extract("Column (\d+)").astype(int)

# df_1.sample(n=7)

#### Drop duplicated Columns, Reorder, Rename, & Sort by Date

In [ ]:
df_2 = df_1.drop(labels=['sitename', 'date'], axis=1)
df_2.shape

In [ ]:
df_2.columns

In [ ]:
col_reorder = ['date_1', 'sitename_1', 'range', 'column', 'lat', 'lon', 'cultivar', 'trait', 'mean']

In [ ]:
df_3 = pd.DataFrame(data=df_2, columns=col_reorder, index=df_2.index)
# df_3.head()

In [ ]:
df_4 = df_3.rename({'date_1': 'date', 'sitename_1': 'sitename', 'mean': 'value'}, axis=1)
# df_4.tail()

In [ ]:
df_5 = df_4.sort_values(by=['date'], ascending=True)
print(df_5.shape)
print(df_4.shape)
# df_5.head()

### III. Drop traits not needed at this time
* dropping `canopy_height` because of the E W subplots - dealing with those values separately

In [ ]:
# df_5.trait.unique()

In [ ]:
traits_to_keep = ['flowering_time', 'flag_leaf_emergence_time', 'light_intensity_PAR', 'aboveground_dry_biomass']

In [ ]:
df_6 = df_5.loc[df_5.trait.isin(traits_to_keep)]
df_6.shape

In [ ]:
df_6.isnull().sum()

In [ ]:
# df_6.head()

In [ ]:
df_7 = df_6.set_index('date')
# df_7.head()

#### Final Steps
* Create `.csv`

In [ ]:
need_to_create_csv = False

if need_to_create_csv:

    timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
    output_filename = f'tall_format_traits_season_4_{timestamp}.csv'.replace(':', '')
    df_7.to_csv(f'data/processed/{output_filename}')